In [1]:
# Instala se precisar
#!pip install selenium pandas webdriver-manager

In [2]:
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from io import StringIO
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

def extrair_ajustes_b3_por_data_final(data_str: str) -> pd.DataFrame:
    """
    Extrai os ajustes do pregão da B3/BMF usando o POST request para o URL correto do iframe.

    Args:
        data_str (str): A data de busca no formato 'DD/MM/AAAA'.

    Returns:
        pd.DataFrame: O DataFrame com os dados da tabela de ajustes, ou um DataFrame vazio.
    """
    
    # NOVO URL DE DESTINO (O URL REAL DENTRO DO IFRAME)
    URL_REAL_BMF = "https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-ajustes-do-pregao-ptBR.asp"
    ID_TABELA = 'tblDadosAjustes'

    # PAYLOAD: Mantemos os campos de data DD/MM/AAAA
    payload = {
        'dData1': data_str, # Data Inicial
        'dData2': data_str  # Data Final
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        # O Referer deve ser o URL do wrapper para simular a navegação
        'Referer': 'https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/derivativos/ajustes-do-pregao/',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Connection': 'keep-alive'
    }

    try:
        # 1. Enviar o POST request para a URL REAL da BMF
        response = requests.post(URL_REAL_BMF, data=payload, headers=headers)
        response.raise_for_status()

        # Debug do Novo Response
        #print(f"Status Code do novo POST: {response.status_code}")
        
        # 2. Analisar o HTML da resposta
        soup = BeautifulSoup(response.text, 'html.parser')
                    
        # 3. Buscar a tabela pelo ID exato
        tabela_html = soup.find('table', {'id': ID_TABELA})
        
        if tabela_html:            
            # Lê o objeto HTML da tabela diretamente
            df_list = pd.read_html(StringIO(str(tabela_html)), encoding='utf-8')
            if df_list:
                return df_list[0] 
            
        # Se a tabela não for encontrada, retorna vazio
        return pd.DataFrame()

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer o request HTTP: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return pd.DataFrame()

def filtrar_por_contrato(df_completo: pd.DataFrame, contratos_siglas: list) -> dict:
    """
    Filtra o DataFrame de ajustes do pregão para exibir apenas os contratos listados nas siglas.

    Args:
        df_completo (pd.DataFrame): O DataFrame completo retornado pela extração.
        contratos_siglas (list(str)): As siglas dos contratos a serem filtrados (ex: 'ICF', 'DOL', 'IND').

    Returns:
        dict(pd.DataFrame): Dicionário de DataFrames contendo apenas os dados dos contratos desejados.
    """
    # Defina o nome da coluna de Contrato
    colunas = ["Mercadoria", "Vencimento", "Preço de ajuste Atual"]
    # Dicionário de contratos
    dict_contratos = {}

    df_completo['Mercadoria'] = df_completo['Mercadoria'].ffill()
    # Itera sob os contratos a serem filtrados
    for sigla in contratos_siglas: 
        # Aplicar o filtro
        df_filtrado = (df_completo[df_completo["Mercadoria"] == sigla])[colunas]

        # Insere no dicionário
        dict_contratos[sigla] = df_filtrado

    return dict_contratos
        

def vencimento_mais_proximo(dict_filtrado: dict, data: datetime, contratos_siglas) -> dict:
    """
    Concatena o preço do contrato de vencimento mais proximo (vamos testar para df_filtrado[0], se não for, temos que usar a regra se vencimento do contrato)

    Args:
        dict_filtrado(dict): O dicionário com os DataFrames filtrados dos contratos negociados nessa data
        data(str): Data dos contratos no dict_filtrado
        contratos_siglas (list(str)): As siglas dos contratos no dict_filtrado.
    
    Returns:
        dict: Dicionário com
            Contrato negocioado: 
            - data: data de negociação
            - valor: valor do contrato de vencimento mais próximo 
    """
    dict_precos_data = {}

    for sigla in contratos_siglas:
        # Dicionário com contrato contendo informações de: Data e Valor do contrato negociado de vencimento mais próximo
        contrato = {
            "Data": data,
            "Valor": dict_filtrado[sigla].head(1)["Preço de ajuste Atual"].item()
        }
        # Associa isso a sigla do contrato
        dict_precos_data[sigla] = contrato

    return dict_precos_data

In [29]:
MAX_THREADS = 32 # Número de requisições simultâneas

# --- Exemplo de Uso com Paralelismo ---
FORMATO_DATA = "%d/%m/%Y"
DATA_INICIAL_DT = datetime.strptime("06/10/2025", FORMATO_DATA)
DATA_FINAL_DT = datetime.strptime("01/01/2010", FORMATO_DATA)
SIGLAS_CONTRATOS = ["ETH - Etanol Hidratado", "BGI - Boi gordo"]

# 1. Gerar a lista de datas a serem processadas
datas_a_processar = []
data_atual = DATA_INICIAL_DT
while data_atual >= DATA_FINAL_DT:
    datas_a_processar.append(data_atual)
    data_atual -= timedelta(days=1)

print(f"Total de {len(datas_a_processar)} dias a serem processados. Iniciando requisições em paralelo...")

# Inicializa as listas de resultados
resultados_etanol = []
resultados_boi = []

# Mapeia as siglas para as listas de resultados
mapa_resultados = {
    "ETH - Etanol Hidratado": resultados_etanol,
    "BGI - Boi gordo": resultados_boi,
}

# 2. Executar em paralelo
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Mapeia a função de extração para cada data, passando a data formatada
    # O 'submit' retorna um objeto Future
    future_to_date = {
        executor.submit(extrair_ajustes_b3_por_data_final, dt.strftime(FORMATO_DATA)): dt
        for dt in datas_a_processar
    }

    # Processa os resultados à medida que eles ficam prontos
    for future in as_completed(future_to_date):
        data_negociacao = future_to_date[future]
        
        try:
            df_ajustes = future.result()
            data_str = data_negociacao.strftime(FORMATO_DATA)
            
            if not df_ajustes.empty:
                print(f"Sucesso ao puxar e processar contratos do dia {data_str}")
                
                # 3. Aplica o filtro e processamento (agora fora do loop HTTP)
                dict_filtrados = filtrar_por_contrato(df_ajustes, SIGLAS_CONTRATOS)
                # Pega os vencimentos mais próximos
                dict_venc_prox = vencimento_mais_proximo(dict_filtrados, data_negociacao, SIGLAS_CONTRATOS)
                
                # 4. Associa cada valor de contrato ao df respectivo
                for sigla in SIGLAS_CONTRATOS:
                    # Verifica se o contrato existe no resultado (evita KeyError)
                    if sigla in dict_venc_prox:
                         mapa_resultados[sigla].append(dict_venc_prox[sigla])
            else:
                print(f"Nenhuma negociação encontrada ou tabela vazia para o dia {data_str}")

        except Exception as exc:
            print(f"Ocorreu um erro ao processar a data {data_negociacao.strftime(FORMATO_DATA)}: {exc}")


# 5. Transforma em DataFrames finais
df_etanol = pd.DataFrame(resultados_etanol)
df_boi = pd.DataFrame(resultados_boi)

display(df_etanol)
display(df_boi)

Total de 5758 dias a serem processados. Iniciando requisições em paralelo...
Nenhuma negociação encontrada ou tabela vazia para o dia 27/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 04/10/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 28/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 21/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 05/10/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 20/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 14/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 07/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 06/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 13/09/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 31/08/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 30/08/2025
Nenhuma negociação encontrada ou tabela vazia para o dia 24/08/2025
Nenhuma negociação encontrada ou tabela

,Data,Valor
0,2025-08-25,"2.784,50"
1,2025-08-22,"2.788,50"
2,2025-09-26,"2.830,50"
3,2025-10-06,"2.846,00"
4,2025-09-02,"2.885,50"
...,...,...
3814,2010-05-21,72500
3815,2010-05-24,72750
3816,2010-05-20,72750
3817,2010-05-18,72750


,Data,Valor
0,2025-08-25,31185
1,2025-08-22,31200
2,2025-09-26,30360
3,2025-10-06,31110
4,2025-09-02,31655
...,...,...
3814,2010-05-21,8118
3815,2010-05-24,8110
3816,2010-05-20,8112
3817,2010-05-18,8044


In [34]:
# Convertendo os Valores para Float com duas casas decimais (Aqui depende de cada contrato, então é importante analisar o que ta vindo como resposta)
df_etanol["Valor"] = (pd.to_numeric(df_etanol['Valor'].astype(str).str.replace(',','', regex=False).str.replace('.','', regex=False), errors='coerce')/100)
df_boi["Valor"] = (pd.to_numeric(df_boi['Valor'], errors='coerce')/100)

In [35]:
# Setando a Data como indice
df_etanol.set_index("Data", inplace=True)
df_boi.set_index("Data", inplace=True)

display(df_etanol)
display(df_boi)

,Valor
Data,
2025-08-25,2784.5
2025-08-22,2788.5
2025-09-26,2830.5
2025-10-06,2846.0
2025-09-02,2885.5
...,...
2010-05-21,725.0
2010-05-24,727.5
2010-05-20,727.5


,Valor
Data,
2025-08-25,311.85
2025-08-22,312.00
2025-09-26,303.60
2025-10-06,311.10
2025-09-02,316.55
...,...
2010-05-21,81.18
2010-05-24,81.10
2010-05-20,81.12


In [6]:
# Vamos comparar com a série do investing
file_path = "data/Arabica Coffee 4_5 Futures Historical Data.csv"
df_investing = pd.read_csv(file_path)
df_investing

,Date,Price,Open,High,Low,Vol.,Change %
0,09/10/2025,498.95,498.95,498.95,498.95,NaN,2.17%
1,09/09/2025,488.35,505.00,505.00,505.00,0.00K,-1.28%
2,09/08/2025,494.70,494.70,494.70,494.70,NaN,3.37%
3,09/06/2025,478.55,478.75,478.55,478.55,0.05K,0.00%
4,09/05/2025,478.55,478.75,480.00,478.75,0.05K,-0.04%
...,...,...,...,...,...,...,...
3888,01/08/2010,177.15,174.15,177.70,174.00,70.75M,1.78%
3889,01/07/2010,174.05,174.50,174.70,171.90,43.89M,-0.23%
3890,01/06/2010,174.45,174.00,175.00,173.00,30.99M,0.23%
3891,01/05/2010,174.05,175.00,175.20,173.50,22.14M,-0.43%


In [ ]:

print("\n--- Iniciando Exportação para CSV (Data como Índice) ---")

# ----------------------------------------------------
# 1. Exportar Etanol
# ----------------------------------------------------
if not df_etanol.empty:
    nome_arquivo_etanol = 'serie_historica_etanol_indice_data.csv'
    
    # 1. Ordena pela Data e define a coluna 'Data' como o índice do DataFrame
    df_etanol_final = df_etanol.sort_values(by='Data', ascending=True)
    
    # 2. Salva. Por padrão, o índice (agora a Data) é salvo.
    df_etanol_final.to_csv(nome_arquivo_etanol, sep=';', decimal=',')
    
    print(f"✅ Etanol salvo em: {nome_arquivo_etanol}")
else:
    print("⚠️ DataFrame do Etanol vazio. Arquivo não criado.")


# ----------------------------------------------------
# 2. Exportar Boi
# ----------------------------------------------------
if not df_boi.empty:
    nome_arquivo_boi = 'serie_historica_boi_indice_data.csv'
    
    df_boi_final = df_boi.sort_values(by='Data', ascending=True)
    df_boi_final.to_csv(nome_arquivo_boi, sep=';', decimal=',')
    
    print(f"✅ Boi salvo em: {nome_arquivo_boi}")
else:
    print("⚠️ DataFrame do Boi vazio. Arquivo não criado.")

print("\nExportação concluída.")


--- Iniciando Exportação para CSV (Data como Índice) ---
✅ Etanol salvo em: serie_historica_etanol_indice_data.csv
✅ Boi salvo em: serie_historica_boi_indice_data.csv

Exportação concluída.
